In [1]:
import modin.pandas as md
import time
import random
import pandas as pd

In [2]:
mdf = md.read_csv('random_people.csv')

In [3]:
mdf.head()

,name,surname,salary
0,Henry,Joneson,5000
1,Albert,Goodman,10000
2,William,Goodman,10000
3,John,Joneson,10000
4,Albert,Black,10000


In [4]:
df = pd.read_csv('random_people.csv')

In [5]:
df.head()

,name,surname,salary
0,Henry,Joneson,5000
1,Albert,Goodman,10000
2,William,Goodman,10000
3,John,Joneson,10000
4,Albert,Black,10000


In [6]:
def benchmark(function, df):
    start = time.time()
    function(df)
    end = time.time()
    print("{} seconds for '{}' '{}'".format((end - start), df.__module__.split('.')[0], function.__name__))

In [7]:
def add_bonus(df):
    df['bonus'] = df['salary'] * 0.2

#### simple test for adding new column 

In [8]:
benchmark(add_bonus, mdf)
benchmark(add_bonus, df)

0.06459903717041016 seconds for 'modin' 'add_bonus'
0.0042266845703125 seconds for 'pandas' 'add_bonus'


#### create large dataframe 

In [9]:
df = pd.concat([df for _ in range(1000)])
df = pd.concat([df for _ in range(500)])

df.head()

,name,surname,salary,bonus
0,Henry,Joneson,5000,1000.0
1,Albert,Goodman,10000,2000.0
2,William,Goodman,10000,2000.0
3,John,Joneson,10000,2000.0
4,Albert,Black,10000,2000.0


In [10]:
df.shape

(25000000, 4)

In [11]:
df.to_csv('./large_file.csv', index=False)

In [12]:
mdf = md.read_csv('./large_file.csv')

mdf.head()

,name,surname,salary,bonus
0,Henry,Joneson,5000,1000.0
1,Albert,Goodman,10000,2000.0
2,William,Goodman,10000,2000.0
3,John,Joneson,10000,2000.0
4,Albert,Black,10000,2000.0


In [13]:
mdf.shape

(25000000, 4)

In [14]:
def get_mean(df):
    x = df.salary.mean()
    return x

def get_max(df):
    x = df.salary.max()
    return x

def get_sum(df):
    x = df.salary.sum()
    return x

def filter_test(df):
    x = df[df['salary']>5000]
    return x

def run_benchmarks():
    for i,f in enumerate([
#                           add_bonus,
                          get_mean,
                          get_max,
                          get_sum,
                          filter_test
                         ]):
        benchmark(f, mdf)
        benchmark(f, df)
                          

In [15]:
def f(x):
    return (13*x+5)%7

def apply_func(df):
    df['random']= df['salary'].apply(f)
    
def value_count(df):
    x = df.salary.value_counts()
    return x

In [16]:
run_benchmarks()

9.761027097702026 seconds for 'modin' 'get_mean'
0.669762134552002 seconds for 'pandas' 'get_mean'
8.905770063400269 seconds for 'modin' 'get_max'
0.4076240062713623 seconds for 'pandas' 'get_max'
8.870067834854126 seconds for 'modin' 'get_sum'
0.30697011947631836 seconds for 'pandas' 'get_sum'
12.855278253555298 seconds for 'modin' 'filter_test'
3.8595430850982666 seconds for 'pandas' 'filter_test'


In [17]:
benchmark(apply_func, mdf)
benchmark(apply_func, df)
benchmark(value_count, mdf)
benchmark(value_count, df)

19.127413034439087 seconds for 'modin' 'apply_func'
18.082404851913452 seconds for 'pandas' 'apply_func'


To request implementation, send an email to feature_requests@modin.org.


25.079288959503174 seconds for 'modin' 'value_count'
0.7049682140350342 seconds for 'pandas' 'value_count'


In [19]:
mdf.salary.value_counts()

10000    8000000
12000    4000000
11000    3500000
9500     3000000
13500    2500000
5500     2500000
5000     1500000
dtype: int64

In [ ]:
df.salary.value_counts()